In [1]:
# imports
import dspy
import os
from dotenv import load_dotenv

/Users/spartan/Projects/masters/course257/context-based-hate-moderation/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv('../local.env')

True

In [3]:
print(os.environ['GROQ_API_KEY'])
GROQ_MODEL_LARGE = os.environ['GROQ_MODEL_LARGE']

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


In [4]:
GROQ_MODEL_LARGE

'llama-3.1-70b-versatile'

In [5]:
llm = dspy.LM(model=f"groq/{GROQ_MODEL_LARGE}", max_tokens=8000, cache=False)
dspy.configure(lm=llm, temperature=0)

In [6]:
class ContextHateAnalysis(dspy.Signature):
    """
    Using the context and the comment, understand the whole meaning of the message and tell me if the comment conveys hate or not. 
    There are certain conditions to be considered before determining if the comment is hateful or not.
    - The comment might or might not contain the hatefull words.
    - If the comment contains the hatefull words, it might not be hateful. Its just the word used in a posive context and never conveyed hateful message.
    - Even if the comment doesnt contain hatefull words in it, it doesnt mean it is not hateful. The comment might be hateful in a way that it is conveying with just non hateful words.
    - Sarcasm and irony are also to be considered as hate.
    - Classify as hateful if the comment is conveying hate even a little. Answer sensitively.
    
    Understand the whole context and determine the result.
    
    Output if the comment is Hateful or not. "True" if hateful, "False" if not hateful.
    """

    comment: str = dspy.InputField()
    context: str = dspy.InputField()
    output = dspy.OutputField(
        desc="""Strictly Tell "True" if hateful, "False" if not hateful.""",
    )

In [16]:
class ContextHateExplaination(dspy.Signature):
    """
    Using the context and the comment, understand the whole meaning of the message and tell me if the comment conveys hate or not. 
    There are certain conditions to be considered before determining if the comment is hateful or not.
    - The comment might or might not contain the hatefull words.
    - If the comment contains the hatefull words, it might not be hateful. Its just the word used in a posive context and never conveyed hateful message.
    - Even if the comment doesnt contain hatefull words in it, it doesnt mean it is not hateful. The comment might be hateful in a way that it is conveying with just non hateful words.
    - Unserstand the sarcasm and irony in the comment. Sometimes the comment might be sarcastic and not hateful.
    
    Understand the whole context and determine the result. There might be simple terms which can still portray hate.
    
    Tell "False" if not hateful. 
    If hateful, provide the warning message in a format: "The comment is hateful as it is targetting [specific person/group/community] in a [specific way]."
    """

    comment: str = dspy.InputField()
    context: str = dspy.InputField()
    output = dspy.OutputField(
        desc="""Strictly Tell "False" if not hateful. If hateful, provide the warning message.""",
    )

In [17]:
result = dspy.ChainOfThought(ContextHateExplaination, max_retries=3).forward(
    context="",
    comment="I dont like him.",
)
output = result.output
print(result.output)

False


In [18]:
result = dspy.ChainOfThought(ContextHateExplaination, max_retries=3).forward(
    context="That was a great movie",
    comment="They killed it.",
)
# output1 = result2.output
print(result.output)

False
